In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from src.models.ts2vec_src.ts2vec import TS2Vec
from src.data.preprocessing import read_data, preprocessing, data_to_np_tensor

%load_ext autoreload
%autoreload 2

/tmp/ipykernel_8647/4076450065.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# IDE

In [12]:
start_date = '2023-12-20'
end_date = '2023-12-20'

data = read_data('data/all_tickers.csv')

mask_lower = data['Datetime'].dt.date >= pd.Timestamp(start_date).date()
mask_upper = data['Datetime'].dt.date < pd.Timestamp(end_date).date()
data = data[(mask_lower) & (mask_upper)]

KeyboardInterrupt: 

In [ ]:

df_tickers = data.pivot(index='Datetime', columns='Stock', values=value_col)


# Preprocessing

In [7]:
data = read_data('data/all_tickers.csv')
train_data = preprocessing(
    data, 
    ['Open', 'High', 'Low', 'Close', 'Volume'],         
    start_date = '2023-12-15',
    end_date = '2023-12-21'
)

test_data = preprocessing(
    data, 
    ['Open', 'High', 'Low', 'Close', 'Volume'],          
    start_date = '2023-12-21',
    end_date = '2023-12-22',
    tickers_save = train_data['Close'].columns
)

N_saved_stocks  114
NAN in data  0
N_saved_stocks  114
NAN in data  0
N_saved_stocks  114
NAN in data  0
N_saved_stocks  114
NAN in data  0
N_saved_stocks  114
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0


In [8]:
for feature in ['Open', 'High', 'Low', 'Close', 'Volume']:
    print(train_data['Close'].shape[1], test_data['Close'].shape[1])

114 114
114 114
114 114
114 114
114 114


In [9]:
train_ts = data_to_np_tensor(train_data)
test_ts = data_to_np_tensor(test_data)

In [10]:
model = TS2Vec(
    input_dims=train_ts.shape[2],
    device=1,
    output_dims=320
)

loss_log = model.fit(
    train_ts,
    verbose=False
)

test_repr = model.encode(test_ts)
#test_repr = model.encode(test_data, encoding_window='full_series')

In [11]:
test_repr = model.encode(test_ts, encoding_window='full_series')
test_repr.shape

(114, 320)

In [14]:
test_repr

array([[151.66946  , 202.70293  ,  90.33598  , ..., 112.1458   ,
          6.0127563,  22.613007 ],
       [197.37054  , 367.42865  , 135.26004  , ..., 148.20499  ,
        -29.922295 ,  47.031876 ],
       [ -8.969391 ,  17.11412  ,   4.0358434, ..., -12.823837 ,
         -9.075836 ,   1.972024 ],
       ...,
       [ 52.176483 ,  90.83186  ,  22.670506 , ...,  25.386154 ,
         -6.0261917,   8.159452 ],
       [  5.8710136,  25.718458 ,  19.747057 , ...,   1.4381132,
         -3.1364307,   6.582671 ],
       [118.346985 , 179.11423  , 162.83815  , ...,  69.087395 ,
         -5.468624 ,  12.737125 ]], dtype=float32)

# Data Transformation

In [4]:
day = '2023-12-20'

df = pd.read_csv('data/all_tickers.csv', index_col = 0)
df['Datetime'] = pd.to_datetime(df['Datetime'])

df_day = df[df['Datetime'].dt.date == pd.Timestamp(day).date()]

df_day.head()

,Date,Time,Open,High,Low,Close,Volume,Datetime,Stock
2800,20231220,235900,138.245,138.35,138.145,138.15,2980,2023-12-20 23:59:00,A
2801,20231220,235800,138.270,138.33,138.230,138.23,950,2023-12-20 23:58:00,A
2802,20231220,235700,138.320,138.32,138.260,138.26,419,2023-12-20 23:57:00,A
2803,20231220,235600,138.310,138.45,138.310,138.36,3626,2023-12-20 23:56:00,A
2804,20231220,235500,138.590,138.59,138.570,138.59,381,2023-12-20 23:55:00,A


In [5]:
df_tickers = df_day.pivot(index='Datetime', columns='Stock', values=['Close', 'Open'])
df_tickers

Close                                                    \
Stock                     A     AAL     AAPL    ABBV    ABNB     ABT      ACN   
Datetime                                                                        
2023-12-20 00:00:00     NaN     NaN      NaN     NaN     NaN     NaN      NaN   
2023-12-20 17:30:00     NaN  14.130  196.955     NaN  145.47  108.04  345.240   
2023-12-20 17:31:00     NaN  14.180  196.830  153.60  145.63  107.95  346.330   
2023-12-20 17:32:00     NaN  14.140  196.970  153.75  144.87  107.70  345.955   
2023-12-20 17:33:00     NaN  14.190  196.880  154.10  145.11  107.87  345.635   
...                     ...     ...      ...     ...     ...     ...      ...   
2023-12-20 23:55:00  138.59  14.010  195.270  152.10  141.18  107.54  340.080   
2023-12-20 23:56:00  138.36  13.995  195.240  151.76  141.18  107.42  340.150   
2023-12-20 23:57:00  138.26  13.985  195.070  151.69  141.00  107.30  339.810   
2023-12-20 23:58:00  138.23  13.985  195.120  151.67  141.05  107.42  339.750   
2023-12-20 23:59:00  138.15  13.985  194.870  151.67  141.10  107.39  339.370   

                                              ...   Open                  \
Stock                   ADBE     ADI     ADM  ...     WY    WYNN     XEL   
Datetime                                      ...                          
2023-12-20 00:00:00      NaN     NaN     NaN  ...    NaN     NaN     NaN   
2023-12-20 17:30:00  606.220     NaN  73.170  ...    NaN  91.570  62.120   
2023-12-20 17:31:00  607.255     NaN     NaN  ...    NaN  91.155     NaN   
2023-12-20 17:32:00  608.610  196.61  73.290  ...  34.40     NaN     NaN   
2023-12-20 17:33:00  609.250     NaN  73.130  ...  34.42     NaN     NaN   
...                      ...     ...     ...  ...    ...     ...     ...   
2023-12-20 23:55:00  597.750  192.39  70.810  ...  34.20  88.750  61.510   
2023-12-20 23:56:00  597.140  192.37  70.705  ...  34.18  88.710  61.415   
2023-12-20 23:57:00  596.310  192.20  70.660  ...  34.15  88.660  61.400   
2023-12-20 23:58:00  596.380  192.25  70.640  ...  34.14  88.520  61.360   
2023-12-20 23:59:00  596.390  192.20  70.670  ...  34.15  88.570  61.340   

                                                                               
Stock                    XOM    XRAY     XYL     YUM     ZBH    ZION      ZTS  
Datetime                                                                       
2023-12-20 00:00:00      NaN     NaN     NaN     NaN     NaN     NaN      NaN  
2023-12-20 17:30:00  103.310     NaN     NaN     NaN  120.23  43.530      NaN  
2023-12-20 17:31:00  103.180  34.665     NaN     NaN  120.24     NaN      NaN  
2023-12-20 17:32:00  103.410     NaN     NaN     NaN  120.36  43.690      NaN  
2023-12-20 17:33:00  103.250     NaN     NaN     NaN  119.93  43.770      NaN  
...                      ...     ...     ...     ...     ...     ...      ...  
2023-12-20 23:55:00  101.440  34.065  109.68  129.32  119.74  42.740  195.210  
2023-12-20 23:56:00  101.310  34.010  109.70  129.12  119.64  42.630  195.050  
2023-12-20 23:57:00  101.180  34.000  109.61  129.05  119.55  42.620  194.790  
2023-12-20 23:58:00  101.140  34.010  109.66  128.99  119.53  42.590  194.655  
2023-12-20 23:59:00  101.205  34.025  109.69  128.96  119.69  42.585  194.750  

[391 rows x 884 columns]

Обработка пропусков

In [6]:
df_tickers.isna().sum().sum()

186582

In [7]:
df_tickers.isna().astype(int).mean()

       Stock
Close  A        0.682864
       AAL      0.240409
       AAPL     0.002558
       ABBV     0.473146
       ABNB     0.248082
                  ...   
Open   XYL      0.762148
       YUM      0.716113
       ZBH      0.590793
       ZION     0.496164
       ZTS      0.693095
Length: 884, dtype: float64

In [8]:
thrs_stocks = 0.3
nan_df = df_tickers.isna().astype(int).mean().to_frame(name='nan_ratio')
tickers_save = nan_df[nan_df['nan_ratio'] < thrs_stocks].index 
print('N_saved_stocks ', len(tickers_save))

df_tickers = df_tickers[tickers_save]

df_tickers = df_tickers.interpolate(method='linear', limit_direction='both')

print('NAN in data ', df_tickers.isna().sum().sum())

N_saved_stocks  218
NAN in data  0


In [9]:
df_tickers

Close                                                    \
Stock                   AAL     AAPL    ABNB     ABT     ADM     AES     AMAT   
Datetime                                                                        
2023-12-20 00:00:00  14.130  196.955  145.47  108.04  73.170  19.280  160.410   
2023-12-20 17:30:00  14.130  196.955  145.47  108.04  73.170  19.280  160.410   
2023-12-20 17:31:00  14.180  196.830  145.63  107.95  73.230  19.295  160.635   
2023-12-20 17:32:00  14.140  196.970  144.87  107.70  73.290  19.170  160.860   
2023-12-20 17:33:00  14.190  196.880  145.11  107.87  73.130  19.290  160.830   
...                     ...      ...     ...     ...     ...     ...      ...   
2023-12-20 23:55:00  14.010  195.270  141.18  107.54  70.810  18.395  157.140   
2023-12-20 23:56:00  13.995  195.240  141.18  107.42  70.705  18.380  157.090   
2023-12-20 23:57:00  13.985  195.070  141.00  107.30  70.660  18.360  156.960   
2023-12-20 23:58:00  13.985  195.120  141.05  107.42  70.640  18.365  156.990   
2023-12-20 23:59:00  13.985  194.870  141.10  107.39  70.670  18.360  156.910   

                                              ...    Open                  \
Stock                    AMD     BAC      BK  ...     USB       V     VFC   
Datetime                                      ...                           
2023-12-20 00:00:00  139.280  33.320  51.490  ...  43.830  260.32  18.580   
2023-12-20 17:30:00  139.280  33.320  51.490  ...  43.830  260.32  18.580   
2023-12-20 17:31:00  139.210  33.390  51.490  ...  43.810  260.44  18.600   
2023-12-20 17:32:00  139.360  33.410  51.500  ...  43.770  260.13  18.490   
2023-12-20 17:33:00  138.980  33.480  51.520  ...  43.830  260.01  18.350   
...                      ...     ...     ...  ...     ...     ...     ...   
2023-12-20 23:55:00  135.790  33.015  51.010  ...  42.640  257.79  17.980   
2023-12-20 23:56:00  135.705  33.005  50.970  ...  42.565  257.59  17.950   
2023-12-20 23:57:00  135.490  32.970  50.925  ...  42.505  257.45  17.920   
2023-12-20 23:58:00  135.550  32.975  50.950  ...  42.490  257.32  17.900   
2023-12-20 23:59:00  135.430  32.970  50.880  ...  42.500  257.35  17.935   

                                                                          \
Stock                    VZ     WBA     WFC       WMB     WMT        XEL   
Datetime                                                                   
2023-12-20 00:00:00  37.520  26.010  49.860  34.97000  155.45  62.120000   
2023-12-20 17:30:00  37.520  26.010  49.860  34.97000  155.45  62.120000   
2023-12-20 17:31:00  37.555  25.990  49.910  34.99000  155.72  62.093333   
2023-12-20 17:32:00  37.550  25.970  49.880  34.97500  155.76  62.066667   
2023-12-20 17:33:00  37.520  25.970  49.870  34.97875  155.68  62.040000   
...                     ...     ...     ...       ...     ...        ...   
2023-12-20 23:55:00  37.430  25.425  49.460  34.70000  153.90  61.510000   
2023-12-20 23:56:00  37.395  25.415  49.390  34.66500  153.83  61.415000   
2023-12-20 23:57:00  37.370  25.400  49.340  34.62500  153.73  61.400000   
2023-12-20 23:58:00  37.340  25.400  49.305  34.61500  153.67  61.360000   
2023-12-20 23:59:00  37.340  25.415  49.330  34.62000  153.76  61.340000   

                              
Stock                    XOM  
Datetime                      
2023-12-20 00:00:00  103.310  
2023-12-20 17:30:00  103.310  
2023-12-20 17:31:00  103.180  
2023-12-20 17:32:00  103.410  
2023-12-20 17:33:00  103.250  
...                      ...  
2023-12-20 23:55:00  101.440  
2023-12-20 23:56:00  101.310  
2023-12-20 23:57:00  101.180  
2023-12-20 23:58:00  101.140  
2023-12-20 23:59:00  101.205  

[391 rows x 218 columns]

# Training embeddings

In [10]:
np_data = df_tickers.values.T
data_shapes = np_data.shape

np_data = np_data.reshape(*data_shapes, 1)
np_data.shape

(218, 391, 1)

In [11]:
train_data = np_data
# (Both train_data and test_data have a shape of n_instances x n_timestamps x n_features)

# Train a TS2Vec model
model = TS2Vec(
    input_dims=1,
    device=1,
    output_dims=320
)

loss_log = model.fit(
    train_data,
    verbose=True
)

Epoch #0: loss=50935.21579214243
Epoch #1: loss=2368.3222233698916
Epoch #2: loss=944.6271784855769
Epoch #3: loss=979.9864067664513
Epoch #4: loss=425.1427776630108
Epoch #5: loss=232.24176025390625
Epoch #6: loss=173.19260670588568
Epoch #7: loss=154.29953853900616
Epoch #8: loss=106.77300409170297
Epoch #9: loss=74.55095195770264
Epoch #10: loss=147.52413881742038
Epoch #11: loss=120.66609764099121
Epoch #12: loss=139.37923416724573
Epoch #13: loss=105.78804074800931
Epoch #14: loss=99.87868529099684


In [12]:
# # Compute timestamp-level representations for test set
# test_repr = model.encode(test_data)  # n_instances x n_timestamps x output_dims

# # Compute instance-level representations for test set
# test_repr = model.encode(test_data, encoding_window='full_series')  # n_instances x output_dims

# # Sliding inference for test set
# test_repr = model.encode(
#     test_data,
#     causal=True,
#     sliding_length=1,
#     sliding_padding=50
# )  # n_instances x n_timestamps x output_dims